## Experimental results : denoising a known covariance matrix

We generate a vector of means and a covariance matrix out of 10 blocks of size 50
each, where off-diagonal elements within each block have a correlation of 0.5.

- This covariance matrix is a stylized representation of a “true” (non-empirical) de-toned correlation
matrix of the S&P 500, where each block is associated with an economic sector.

- Without loss of generality, the variances are drawn from a uniform distribution bounded between
5% and 20%, and the vector of means is drawn from a Normal distribution with mean and standard
deviation equal to the standard deviation from the covariance matrix.

In [112]:
# Import library
import numpy as np
from scipy.linalg import block_diag

#### The real covariance matrix

In [113]:
# Generate our covariance matrix
Nb_blocks = 10
block_size = 50
block_list = []
for i in range(Nb_blocks):
    sd = np.sqrt(np.random.uniform(0.05,0.2,50))
    sd = sd[:,np.newaxis]
    C = sd @ np.transpose(sd) 
    diag = np.diag(C)
    C = C * 0.5
    np.fill_diagonal(C, diag)
    block_list.append(C)

Cov = block_diag(*block_list)


In [114]:
# Generate mean
sd = np.sqrt(np.diag(Cov))
mu = np.random.multivariate_normal(sd, Cov)

In [115]:
# The minimum-variance weight 
ones = np.ones((500,1))
w = np.linalg.inv(Cov) @ ones / (np.transpose(ones) @ np.linalg.inv(Cov) @ ones)

#### The estimated covariance matrix with noise

In [116]:
T = 1000
# Generate observation matrix X
X = np.random.multivariate_normal(mu, Cov, T)
# Estimation without denoising
mu_hat = np.mean(X, axis = 0)
mu_hat = mu_hat[:,np.newaxis]
Cov_hat = 1 / T * np.transpose(X) @ X - mu_hat @ np.transpose(mu_hat)
Cov_hat[Cov_hat < 0] = 0
w_hat = np.linalg.inv(Cov_hat) @ ones / (np.transpose(ones) @ np.linalg.inv(Cov_hat) @ ones)
rmse = np.sqrt(np.mean((w_hat-w)**2))
print(rmse)

0.006142104194258463


#### The denoised covariance matrix

In [117]:
q = Nb_blocks * block_size / T
# Lambda max for a correlation matrix is
lmax = (1 + np.sqrt(q)) ** 2

In [ ]:
diag_cov_hat_inv_sqrt = np.diag(1 / np.sqrt(np.diag(Cov_hat)))
Corr_hat = diag_cov_hat_inv_sqrt @ Cov_hat @ diag_cov_hat_inv_sqrt

In [137]:
eig = np.linalg.eig(Corr_hat)
u_hat = eig.eigenvectors
l_hat = eig.eigenvalues

In [140]:
l_hat[l_hat <= lmax] = np.mean(l_hat[l_hat < lmax])

In [141]:
Corr_cleaned = u_hat @ np.diag(l_hat) @ u_hat.T

In [ ]:
diag_corr = np.diag(1 / np.sqrt(np.diag(Corr_cleaned)))
Corr_cleaned = diag_corr @ Corr_cleaned @ diag_corr

In [147]:
diag_cov_hat_sqrt = np.diag(np.diag(Cov_hat))
Cov_cleaned = diag_cov_hat_sqrt @ Corr_cleaned @ diag_cov_hat_sqrt

In [157]:
w_cleaned = np.linalg.inv(Cov_cleaned) @ ones / (np.transpose(ones) @ np.linalg.inv(Cov_cleaned) @ ones)
rmse_cleaned = np.sqrt(np.mean((w_cleaned-w)**2))
print(rmse_cleaned)

0.0023325564843126915


In [158]:
print(rmse)
print(rmse_cleaned)

0.006142104194258463
0.0023325564843126915
